In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Analyzing Soccer Players').getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Analyzing Soccer Players, master=local[*]) created by getOrCreate at C:\Users\Waleed Saleh\AppData\Local\Temp\ipykernel_15380\2193682778.py:1 

In [3]:
players =spark.read\
              .format('csv')\
              .option('header','true')\
              .load('../../data/player.csv')

In [4]:
players.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)



In [5]:
players.show(10)

+---+-------------+------------------+------------------+-------------------+------+------+
| id|player_api_id|       player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+------------------+------------------+-------------------+------+------+
|  1|       505942|Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|   Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|       Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|     Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|      Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
|  6|        27316|        Aaron Hunt|            158138|1986-09-04 00:00:00|182.88|   161|
|  7|       564793|        Aaron Kuhl|            221280|1996-01-30 00:00:00|172.72|   146|
|  8|        30895|      Aaron Lennon|            152747|1987-04-16 00:00:00| 16

In [6]:
player_attributes =spark.read\
                        .format('csv')\
                        .option('header','true')\
                        .load('../../data/player_attributes.csv')

In [7]:
player_attributes.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- potential: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- attacking_work_rate: string (nullable = true)
 |-- defensive_work_rate: string (nullable = true)
 |-- crossing: string (nullable = true)
 |-- finishing: string (nullable = true)
 |-- heading_accuracy: string (nullable = true)
 |-- short_passing: string (nullable = true)
 |-- volleys: string (nullable = true)
 |-- dribbling: string (nullable = true)
 |-- curve: string (nullable = true)
 |-- free_kick_accuracy: string (nullable = true)
 |-- long_passing: string (nullable = true)
 |-- ball_control: string (nullable = true)
 |-- acceleration: string (nullable = true)
 |-- sprint_speed: string (nullable = true)
 |-- agility: string (nullable = true)
 |-- reactions: string (nullable = true

In [8]:
players.count(),player_attributes.count()

(11060, 183978)

In [9]:
player_attributes\
    .select('player_api_id')\
    .distinct()\
    .count()

11060

In [10]:
players=players.drop('id','player_fifa_api_key')
players.columns

['player_api_id',
 'player_name',
 'player_fifa_api_id',
 'birthday',
 'height',
 'weight']

In [11]:
player_attributes = player_attributes.drop(
    'id',
    'player_fifa_api_key',
    'preferred_foot',
    'attacking_work_rate',
    'defensive_work_rate',
    'crossing',
    'jumping',
    'sprint_speed',
    'balance',
    'aggression',
    'short_passing',
    'potential'
)
player_attributes.columns

['player_fifa_api_id',
 'player_api_id',
 'date',
 'overall_rating',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [12]:
players=players.dropna()
player_attributes =player_attributes.dropna()
player_attributes.count(),players.count()

(181265, 11060)

In [13]:
from pyspark.sql.functions import udf

In [14]:
year_extract_udf = udf(lambda date: date.split('-')[0])
player_attributes = player_attributes.withColumn('year',year_extract_udf(player_attributes.date))

In [15]:
player_attributes = player_attributes.drop('date')

Py4JJavaError: An error occurred while calling o74.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 21.0 failed 1 times, most recent failure: Lost task 3.0 in stage 21.0 (TID 26) (192.168.3.13 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions

In [ ]:
pa_2016 = player_attributes.filter(player_attributes['year'] == 2016)
pa_2016.count()

In [ ]:
pa_2016.select(pa_2016.player_api_id)\
    .distinct()\
    .count()

In [ ]:
pa_striker_2016 = pa_2016.groupBy('player_api_id')\
    .agg({
    'finishing':"avg",
    "shot_power":"avg",
    "acceleration":"avg"
})

In [ ]:
pa_striker_2016.count()

In [ ]:
pa_striker_2016.show(10)

In [ ]:
pa_striker_2016 = pa_striker_2016.withColumnRenamed("avg(finishing)","finishing")\
    .withColumnRenamed("avg(shot_power)","shot_power")\
    .withColumnRenamed("avg(acceleration)","acceleration")

In [ ]:
weight_finishing = 1
weight_shot_power = 2
weight_acceleration = 1

total_weight = weight_finishing + weight_shot_power + weight_acceleration

In [ ]:
strikers = pa_striker_2016.withColumn("striker_grade",
                                      (pa_striker_2016.finishing * weight_finishing +
                                       pa_striker_2016.shot_power * weight_shot_power+
                                       pa_striker_2016.acceleration * weight_acceleration) / total_weight)

In [ ]:
strikers = strikers.drop('finishing',
                         'acceleration',
                         'shot_power'
                         )

In [ ]:
strikers = strikers.filter(strikers.striker_grade > 70)\
    .sort(strikers.striker_grade.desc())
strikers.show(10)

In [ ]:
strikers.count(), players.count()

In [ ]:
striker_details = players.join(strikers, players.player_api_id == strikers.player_api_id)

In [ ]:
striker_details.columns

In [ ]:
striker_details.count()

In [ ]:
striker_details = players.join(strikers, ['player_api_id'])

In [ ]:
striker_details.show(5)

In [ ]:
from pyspark.sql.functions import broadcast

In [ ]:
striker_details = players.select(
    "player_api_id",
    "player_name"
)\
    .join(
    broadcast(strikers),
    ['player_api_id'],
    'inner'
)

In [ ]:
striker_details = striker_details.sort(striker_details.striker_grade.desc())

In [ ]:
striker_details.show(10)

In [ ]:
players.count(), player_attributes.count()

In [ ]:
players_heading_acc = player_attributes.select('player_api_id',
                                               'heading_accuracy')\
    .join(broadcast(players),
          player_attributes.player_api_id == players.player_api_id)

In [ ]:
players_heading_acc.columns

In [ ]:
short_count = spark.sparkContext.accumulator(0)
medium_low_count = spark.sparkContext.accumulator(0)
medium_high_count = spark.sparkContext.accumulator(0)
tall_count = spark.sparkContext.accumulator(0)

In [ ]:
def count_players_by_height(row):
    height = float(row.height)

    if height <= 175:
        short_count.add(1)
    elif 183 >= height > 175:
        medium_low_count.add(1)
    elif 195 >= height > 183:
        medium_high_count.add(1)
    elif height > 195:
        tall_count.add(1)

In [ ]:
players_heading_acc.foreach(lambda x: count_players_by_height(x))

In [ ]:
all_players = [short_count.value,
               medium_low_count.value,
               medium_high_count.value,
               tall_count.value]
all_players

In [ ]:
short_ha_count = spark.sparkContext.accumulator(0)
medium_low_ha_count = spark.sparkContext.accumulator(0)
medium_high_ha_count = spark.sparkContext.accumulator(0)
tall_ha_count = spark.sparkContext.accumulator(0)

In [ ]:
def count_players_by_height_and_heading_accuracy(row, threshold_score):

    height = float(row.height)
    ha = float(row.heading_accuracy)

    if ha <= threshold_score:
        return

    if height <= 175:
        short_ha_count.add(1)
    elif 183 >= height > 175:
        medium_low_ha_count.add(1)
    elif 195 >= height > 183:
        medium_high_ha_count.add(1)
    elif height > 195:
        tall_ha_count.add(1)

In [ ]:
players_heading_acc.foreach(lambda x: count_players_by_height_and_heading_accuracy(x, 60))

In [ ]:
all_players_above_threshold = [short_ha_count.value,
                               medium_low_ha_count.value,
                               medium_high_ha_count.value,
                               tall_ha_count.value]

all_players_above_threshold

In [ ]:
percentage_values = [short_ha_count.value / short_count.value *100,
                     medium_low_ha_count.value / medium_low_count.value *100,
                     medium_high_ha_count.value / medium_high_count.value *100,
                     tall_ha_count.value / tall_count.value *100
                     ]

percentage_values

In [6]:
from pyspark.accumulators import AccumulatorParam

class VectorAccumulatorParam(AccumulatorParam):

    def zero(self, value):
        return [0.0] * len(value)

    def addInPlace(self, v1, v2):
        for i in range(len(v1)):
            v1[i] += v2[i]

        return v1

In [7]:
sc = spark.sparkContext
vector_accum = sc.accumulator([10.0, 20.0, 30.0], VectorAccumulatorParam())

vector_accum.value

[10.0, 20.0, 30.0]

In [ ]:
vector_accum += [1, 2, 3]

vector_accum.value

In [ ]:
pa_2016.columns

In [ ]:
pa_2016.select("player_api_id", "overall_rating")\
    .coalesce(1)\
    .write\
    .option("header", "true")\
    .csv("players_overall.csv")

In [ ]:
pa_2016.select("player_api_id", "overall_rating")\
    .write\
    .json("players_overall.json")

In [ ]:
from pyspark.accumulators import AccumulatorParam

class VectorAccumulatorParam(AccumulatorParam):

    def zero(self, value):
        return [0.0] * len(value)

    def addInPlace(self, v1, v2):
        for i in range(len(v1)):
            v1[i] += v2[i]

        return v1

In [ ]:
vector_accum = sc.accumulator([10.0, 20.0, 30.0], VectorAccumulatorParam())

vector_accum.value

In [ ]:
vector_accum += [1, 2, 3]

vector_accum.value

In [ ]:
valuesA = [('John', 100000), ('James', 150000), ('Emily', 65000), ('Nina', 200000)]

tableA = spark.createDataFrame(valuesA, ['name', 'salary'])

In [ ]:
tableA.show()

In [ ]:
valuesB = [('James', 2), ('Emily',3), ('Darth Vader', 5), ('Princess Leia', 6),]

tableB = spark.createDataFrame(valuesB, ['name', 'employee_id'])

In [ ]:
tableB.show()

In [ ]:
inner_join = tableA.join(tableB, tableA.name == tableB.name)
inner_join.show()

In [ ]:
left_join = tableA.join(tableB, tableA.name == tableB.name, how='left')
left_join.show()

In [ ]:
right_join = tableA.join(tableB, tableA.name == tableB.name, how='right')
right_join.show()

In [ ]:
full_outer_join = tableA.join(tableB, tableA.name == tableB.name, how='full')
full_outer_join.show()